In [1]:
import talib
import pandas as pd
import numpy as np
from openapi_client import openapi
import json as js
import datetime
from pandas_datareader import data as pdr

C:\Users\artam\anaconda3_32\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 112 from C header, got 124 from PyObject
  return f(*args, **kwds)


In [59]:
# Выгрузим историю по дням интересующего тикера 

Ticker = 'CAT'

In [60]:
aapl = pdr.get_data_yahoo(Ticker, 
                                  start=datetime.datetime(2020,1,1), 
                                  end=datetime.datetime.today().date()).sort_values(by='Date').reset_index()
aapl['Tick'] = Ticker                           
aapl['Year'] = pd.DatetimeIndex(aapl['Date']).year
aapl['Month'] = pd.DatetimeIndex(aapl['Date']).month
        
        # Расчет показателей
        
        # Дельта между закрытием и открытием

aapl['diff_today'] = round((aapl['Close']-aapl['Open'])/aapl['Close']*100,2)

        # Дельта между закрытием и открытием %

aapl['diff_percent_today'] = round((aapl['Close']-aapl['Open'])/aapl['Close']*100,2)


        # Daily returns

        # Дельта между открытием сегодня и открытием вчера %

aapl['daily_open_perc'] = round((aapl['Open']-  aapl['Open'].shift(1))/ aapl['Open']*100,2)

        # Дельта между закрытием сегодня и закрытием вчера %

aapl['daily_close_perc'] = round((aapl['Close']- aapl['Close'].shift(1))/aapl['Close']*100,2)

        # Закрытие вчера 
aapl['prev_close'] =aapl['Close'].shift(1)
        
        #RSI
        
aapl['RSI_14']= talib.RSI(aapl['Adj Close'],14)
        
        #Volatility
        
aapl['Volatility'] = talib.ATR(aapl['High'],aapl['Low'], aapl['Close'], timeperiod=14)
        


In [7]:
aapl.head(5)

,Date,High,Low,Open,Close,Volume,Adj Close,Tick,Year,Month,diff_today,diff_percent_today,daily_open_perc,daily_close_perc,prev_close,RSI_14,Volatility
0,2020-01-02,333.350006,327.700012,328.549988,333.320007,4544400.0,331.348572,BA,2020,1,1.43,1.43,NaN,NaN,NaN,NaN,NaN
1,2020-01-03,334.890015,330.299988,330.630005,332.760010,3875900.0,330.791901,BA,2020,1,0.64,0.64,0.63,-0.17,333.320007,NaN,NaN
2,2020-01-06,334.859985,327.880005,329.299988,333.739990,5355000.0,331.766083,BA,2020,1,1.33,1.33,-0.40,0.29,332.760010,NaN,NaN
3,2020-01-07,344.190002,330.709991,334.260010,337.279999,9898600.0,335.285156,BA,2020,1,0.90,0.90,1.48,1.05,333.739990,NaN,NaN
4,2020-01-08,334.029999,329.600006,332.399994,331.369995,8239200.0,329.410095,BA,2020,1,-0.31,-0.31,-0.56,-1.78,337.279999,NaN,NaN


In [10]:
aapl['Action']=1

In [53]:
aapl['Action']= np.where((aapl['RSI_14'])<30, 'Buy','None')


In [61]:
aapl[(aapl['RSI_14'])<30]

,Date,High,Low,Open,Close,Volume,Adj Close,Tick,Year,Month,diff_today,diff_percent_today,daily_open_perc,daily_close_perc,prev_close,RSI_14,Volatility
15,2020-01-24,142.699997,139.259995,142.550003,140.380005,2526200.0,138.068878,CAT,2020,1,-1.55,-1.55,0.52,-1.69,142.750000,26.701960,2.346531
16,2020-01-27,137.809998,135.570007,136.899994,135.729996,2920500.0,133.495438,CAT,2020,1,-0.86,-0.86,-4.13,-3.43,140.380005,21.014014,2.522493
17,2020-01-28,137.600006,135.619995,137.020004,136.740005,2763800.0,134.488815,CAT,2020,1,-0.20,-0.20,0.09,0.74,135.729996,24.762885,2.483744
18,2020-01-29,137.759995,135.669998,137.589996,135.779999,2620000.0,133.544601,CAT,2020,1,-1.33,-1.33,0.41,-0.71,136.740005,23.615552,2.455619
19,2020-01-30,135.970001,132.910004,134.759995,135.369995,3862600.0,133.141357,CAT,2020,1,0.45,0.45,-2.10,-0.30,135.779999,23.122820,2.498789
20,2020-01-31,134.990005,130.539993,133.820007,131.350006,6785700.0,129.187546,CAT,2020,1,-1.88,-1.88,-0.70,-3.06,135.369995,18.948226,2.665304
21,2020-02-03,133.160004,129.300003,131.440002,129.770004,4385300.0,127.633560,CAT,2020,2,-1.29,-1.29,-1.81,-1.22,131.350006,17.603056,2.750640
38,2020-02-27,128.529999,123.220001,125.599998,123.269997,7369000.0,121.240570,CAT,2020,2,-1.89,-1.89,-3.22,-4.04,128.250000,24.707584,3.268198
39,2020-02-28,124.239998,119.029999,119.570000,124.239998,8301900.0,122.194603,CAT,2020,2,3.76,3.76,-5.04,0.78,123.269997,27.426835,3.406898
45,2020-03-09,112.989998,104.029999,112.989998,104.070000,8636200.0,102.356667,CAT,2020,3,-8.57,-8.57,-5.32,-16.66,121.410004,20.174010,4.739171


In [36]:
aapl[['Date','Close','RSI_14','Action_2']]

,Date,Close,RSI_14,Action_2
0,2020-01-02,333.320007,NaN,Sell
1,2020-01-03,332.760010,NaN,Sell
2,2020-01-06,333.739990,NaN,Sell
3,2020-01-07,337.279999,NaN,Sell
4,2020-01-08,331.369995,NaN,Sell
...,...,...,...,...
191,2020-10-05,171.199997,56.503534,Sell
192,2020-10-06,159.539993,45.649787,Sell
193,2020-10-07,164.610001,50.135108,Sell
194,2020-10-08,168.000000,52.932110,Sell


In [22]:
aapl[(aapl['Action_1']=='Buy')]

,Date,High,Low,Open,Close,Volume,Adj Close,Tick,Year,Month,diff_today,diff_percent_today,daily_open_perc,daily_close_perc,prev_close,RSI_14,Volatility,Action,Action_1


In [19]:
aapl[(aapl['RSI_14']>70)]

,Date,High,Low,Open,Close,Volume,Adj Close,Tick,Year,Month,diff_today,diff_percent_today,daily_open_perc,daily_close_perc,prev_close,RSI_14,Volatility,Action,Action_1
106,2020-06-04,188.600006,177.399994,181.279999,184.300003,67155600.0,184.300003,BA,2020,6,1.64,1.64,13.34,6.04,173.160004,72.630939,10.271745,Else,Else
107,2020-06-05,218.789993,197.360001,205.000000,205.429993,99846000.0,205.429993,BA,2020,6,0.21,0.21,11.57,10.29,184.300003,78.107155,12.001620,Else,Else
108,2020-06-08,234.199997,218.929993,222.940002,230.500000,80902800.0,230.500000,BA,2020,6,3.28,3.28,8.05,10.88,205.429993,82.564653,13.199361,Else,Else
109,2020-06-09,227.000000,213.080002,222.139999,216.740005,66688700.0,216.740005,BA,2020,6,-2.49,-2.49,-0.36,-6.35,230.500000,73.695568,13.500835,Else,Else
